# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [1]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [5]:
# 4.74 * mu * Ro = VLSR + vsun
def VLSR(Ro, mu=6.379, vsun=12.24*u.km/u.s):
    """
        This function will compute the velocity at the local standard of rest
            VLSR = 4.74 * mu * Ro - vsun
        Inputs: Ro (astropy units kpc) is the distance from the sun to the galactic center
                mu is the proper motion of Sag A* (mas/yr)
                    default is from Reid & Brunthaler 2004
                vsun (astropy units km/s) is the peculiar motion of the sun in the v direction (Schronrich+2010)
        Outputs: VLSR (astropy units km/s) is the local standard of rest
    """
    VLSR = 4.74 * mu * (Ro/u.kpc)*u.km/u.s - vsun
    return VLSR

In [7]:
# different values of the distance to the GC
RoReid = 8.34*u.kpc # Reid + 2014
RoAbuter = 8.178*u.kpc # GRAVITY Abuter + 2019
RoSparke = 7.9*u.kpc # Sparke & Gallagher Text

In [9]:
# compute VLSR using Reid 2014
VLSR_Reid = VLSR(RoReid)
print(VLSR_Reid)

239.9320764 km / s


In [15]:
# compute VLSR using GRAVITY Abuter + 2019
VLSR_Abuter = VLSR(RoAbuter)
print(np.round(VLSR_Abuter))

235.0 km / s


In [13]:
# compute VLSR using Sparke & Gallagher Text
VLSR_Sparke = VLSR(RoSparke)
print(VLSR_Sparke)

226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [21]:
# orbital period = 2piR/V
def TorbSun(Ro, Vc):
    '''
        This is a function that computes the orbital period of the sun
            T = 2 pi R / V
        Inputs: Ro (astropy quantity) is the distance to the galactic center in kpc
                Vc (astropy quantity) is the velocity of the sun in the v direction in km/s
        Outputs: T (astropy quantity) is the orbital peropd in Gyr
    '''
    # converting Vc to kpc/Gyr
    VkpcGyr = Vc.to(u.kpc/u.Gyr)
    # calculating the orbital period
    T = 2 * np.pi * Ro / VkpcGyr

    return T

In [23]:
VsunPec = 12.24*u.km/u.s # peculiar motion

In [25]:
# total motion of the sun in the v direction
Vsun = VLSR_Abuter + VsunPec 

In [29]:
# orbital period of the sun
T_Abuter = TorbSun(RoAbuter, Vsun)
print(np.round(T_Abuter,3))

0.203 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [33]:
AgeUniverse = 13.8*u.Gyr
# calculates the number of rotations of the sun around the GC
NumRotations = AgeUniverse/T_Abuter
print(NumRotations)

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [35]:
print(const.G)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018


In [39]:
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
print(Grav)

4.498502151469554e-06 kpc3 / (solMass Gyr2)


In [41]:
# density profile: rho = VLSR^2 / (4 * pi * G * R^2)
# mass (r) = integrate rho dV
#            integrate rho 4*pi*r^2*dr
#            integrate VLSR^2 / (4*pi*G*r^2) * (4*pi*r^2) dr
#            integrate VLSR^2 dr
#            VLSR^2 / G * r

def massIso(r, VLSR):
    '''
        This function computes the dark matter mass enclosed within a given distance r assuming and isothermal sphere model
            M(r) = VLSR^2 / G * r
        Inputs: r (astropy quantity) is the distance from the galactic center in kpc
                VLSR (astropy quantity) is the velocity at the local standard of rest in km/s
        Outputs: M (astropy quantity) is the mass enclosed within r (Msun)
    '''
    # converting VLSR to kpc/Gyr
    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr)
    # isothermal sphere mass profile
    M = ((VLSRkpcGyr**2)/Grav)*r 
    
    return M

In [47]:
# compute the mass enclosed within Ro (Gravity Collab)
# mass of the isothermal sphere at the radius of the sun
mIsoSolar = massIso(RoAbuter, VLSR_Abuter)
print(f"{mIsoSolar:.2e}")

1.05e+11 solMass


In [49]:
# compute the mass enclosed within 260 kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
print(f"{mIso260:.2e}")

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [57]:
# potential for a Hernquist sphere
# phi = -G*M / (r+a)

# escape speed become:
#  vesc^2 = 2 * G * M / (r + a)

# rearrange for M:
#  M = vesc^2 / 2 / G * (r + a)

def massHernVesc(vesc, r, a=30*u.kpc):
    '''
        This function determines the total dark matter mass needed given an escape speed, assuming a Hernquist profile
            M = vesc^2 / 2 / G * (r + a)
        inputs: vesc (astropy quantity) is the escape speed (or speed of galaxy satellite) in km/s
                a (astropy quantity) is the Hernquist scaling in kpc
                    default value of 30 kpc
                r (astropy quantity) is the distance from the galactic center in kpc
        outputs: M (astropy quantity) is the mass within r in Msun
    '''
    # convert vesc to kpc/Gyr
    vescKpcGyr = vesc.to(u.kpc/u.Gyr)
    M = vescKpcGyr ** 2 / 2 / (Grav) * (a + r)
    
    return M

In [59]:
# speed of leo I Sohn et al.
Vleo = 196*u.km/u.s
r = 260*u.kpc

In [63]:
MLeoI = massHernVesc(Vleo,r)
print(f"{MLeoI:.2e}")

1.30e+12 solMass


In [65]:
mIso260/MLeoI

<Quantity 2.57842045>